# Importing Libraries & Reading JSON

In [1]:
import sys
import os
import subprocess
import json
import itertools
import re

json_list = json.load(open('parameters_json_file.json', 'r'))

data = json_list["data"]
parameters = json_list["parameters"]
fixed_parameters = parameters["fixed"]
iterable_parameters = parameters["iterable"]
#print(iterable_parameters)
#print(log_values)

{'rnn_dim': [100, 150, 200, 300], 'char_rnn': [25, 50], 'word_min_freq': [0, 5, 10], 'train_batch_size': [4, 8, 16]}


## Parameters Combinations

In [ ]:
listofvalues=iterable_parameters.values()
args_list = []
for it_key in iterable_parameters:
    print(it_key, "\t", iterable_parameters[it_key])
    args_list.append(it_key)
parameter_combinations = list(itertools.product(*listofvalues))
print("\nNumber of combinations to try:", len(parameter_combinations))


rnn_dim 	 [100, 150, 200, 300]
char_rnn 	 [25, 50]
word_min_freq 	 [0, 5, 10]
train_batch_size 	 [4, 8, 16]

Number of combinations to try: 72


## Class: Command - add_arg()

In [3]:
class Command_string:
    def __init__(self, initial_string = "python"):
        self.string = initial_string
    
    def change(self, new_string):
        self.string = new_string
    
    def add_arg_str(self, string):
        self.string += " " + string
        
    def add_arg_dic(self, key, dic):
        self.string += " --" + key + " " + str(dic[key])
    
    def add_arg_val(self, key, val):
        self.string += " --" + key + " " + str(val)

## Commands String

### Fixed string part

In [4]:
fixed_command = Command_string()
fixed_command.add_arg_str(json_list["train.py"])
for path_key in data:
    fixed_command.add_arg_dic(path_key, data)
for fixed_par_key in fixed_parameters:
    fixed_command.add_arg_dic(fixed_par_key, fixed_parameters)
#print(fixed_command.string)

python ~/work/model/train.py --data_path ~/work/data/CRF --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --emb_path ~/work/data/embedding/pretrained_embedding.txt --char_emb_dim 25 --word_emb_dim 100 --val_batch_size 64 --max_epochs 100


### Changing common part (due to arguments combinations) --> list of strings

In [5]:
common_list = []
tmp_command = Command_string()
name = []
checkpoint_dirs = []

for comb in parameter_combinations:
    tmp_command.change(fixed_command.string)
    comb_num = parameter_combinations.index(comb)
    name.append("training_" + str(comb_num) + "_")
    for it_par_key in iterable_parameters:
        key_index = list(iterable_parameters.keys()).index(it_par_key)
        tmp_command.add_arg_val(it_par_key, comb[key_index])             # adds --parameterName parameterValue
        name[comb_num] += "__" + it_par_key + "_" + str(comb[key_index]) # adds __parameterName_parameterValue
    common_list.append(tmp_command.string)

### Showing one command

In [6]:
print("List length:", len(common_list))
index_to_show = 3
#print("List command %d (%d/%d):\n%s" % (index_to_show, index_to_show+1, len(common_list), common_list[index_to_show]))
print(name[3])

List length: 72
training_3___rnn_dim_100__char_rnn_25__word_min_freq_5__train_batch_size_4


## Test command

In [8]:
test_command = Command_string()
test_list = []

rang = json_list["test_range"]
for training in range(rang[0],rang[1]+1):
    file_name = json_list["sorted_logs"] + "/" + "sorted_log_" + str(training) + ".table" #Change str(3) to str(training)
    #print(file_name)
    log_file = open(file_name, 'r')
    lines = log_file.readlines()#.replace('\n', '')
    log_values = []
    models = []
    for x in lines:
        log_values.append(re.split(r'\t+', x.rstrip('\t').rstrip('\n')))
        models.append(log_values[lines.index(x)][0])
    log_file.close()
    
    for top_num in range(0,json_list["N_best"]):
        test_command.change(common_list[training])
        test_command.add_arg_val("load_check_point", json_list["checkpoints"] + "/" + name[training] \
                                 + "/" + log_values[top_num][0])
        prediction_str = json_list["predictions"] + "/" + "training_" + str(training) + "__top_" + str(top_num + 1) + \
                         "__epoch_" + str(log_values[top_num][1]) + ".prediction"
        test_command.add_arg_val("test_to_file", prediction_str)
        test_list.append(test_command.string)

sorted_log_3.table
sorted_log_3.table
sorted_log_3.table
sorted_log_3.table
sorted_log_3.table
sorted_log_3.table


In [9]:
print(test_list[2*json_list["N_best"]+0])
#print(json_list["checkpoints"] + "/" + name[comb_num] + "/" + log_values[top_num][0])

python ~/work/model/train.py --data_path ~/work/data/CRF --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --emb_path ~/work/data/embedding/pretrained_embedding.txt --char_emb_dim 25 --word_emb_dim 100 --val_batch_size 64 --max_epochs 100 --rnn_dim 100 --char_rnn 25 --word_min_freq 5 --train_batch_size 4 --load_check_point ~/work/checkpoints/training_3___rnn_dim_100__char_rnn_25__word_min_freq_5__train_batch_size_4/epoch_91_valscore_checkpoint_0.9829214386176411_wc_embedding_lstm_crf.model --test_to_file ~/work/predictions/training_3__top_1__epoch_91.prediction


# Output Files

## Test file

In [11]:
test_file = open(json_list["output_test"], 'w')
for line in test_list:
    test_file.write(line + "\n")
test_file.close()

In [12]:
test_file = open(json_list["output_test"], 'r')
print("Test file:", json_list["output_test"], "\n\n", test_file.read())
test_file.close()

Test file: test_commands.lst 

 python ~/work/model/train.py --data_path ~/work/data/CRF --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --emb_path ~/work/data/embedding/pretrained_embedding.txt --char_emb_dim 25 --word_emb_dim 100 --val_batch_size 64 --max_epochs 100 --rnn_dim 100 --char_rnn 25 --word_min_freq 0 --train_batch_size 8 --load_check_point ~/work/checkpoints/training_1___rnn_dim_100__char_rnn_25__word_min_freq_0__train_batch_size_8/epoch_91_valscore_checkpoint_0.9829214386176411_wc_embedding_lstm_crf.model --test_to_file ~/work/predictions/training_1__top_1__epoch_91.prediction
python ~/work/model/train.py --data_path ~/work/data/CRF --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --emb_path ~/work/data/embedding/pretrained_embedding.txt --char_emb_dim 25 --word_emb_dim 100 --val_batch_size 64 --max_epochs 100 --rnn_dim 100 --char_rnn 25 --word_min_freq 0 --train_batch_size 8 --load_check_point

## Test File

In [17]:
path = "~/work/checkpoints"+"/"+name[1]
print(path)
os.chdir(path)
pwd = subprocess.check_output("pwd")
print(pwd.decode("utf-8"))
cmd = json_list["sort_chekpoints_path"] + " -H " + str(json_list["N_best"]) + " " \
      + json_list["checkpoints"] + "/" + name[1] + " | cut -f 3"
print(cmd)
best_epochs = subprocess.check_output(cmd)
print(best_epochs.decode("utf-8"))

~/work/checkpoints/training_1___char_rnn_25__rnn_dim_100__word_min_freq_0__train_batch_size_8


FileNotFoundError: [Errno 2] No such file or directory: '~/work/checkpoints/training_1___char_rnn_25__rnn_dim_100__word_min_freq_0__train_batch_size_8'

In [ ]:
name = []
for i in range(0,10):
    name.append(str(i))
name2 = name.copy()
name2.sort(reverse=True)
print(name, name2)

## Training command

In [11]:
train_command = Command_string()
train_list = common_list.copy()
for comb in parameter_combinations:
    train_command.change("")
    comb_num = parameter_combinations.index(comb)
    train_command.add_arg_val("checkpoint", json_list["checkpoints"] + "/" + name[comb_num])
    train_command.add_arg_str(" | tee -a " + json_list["logs"] + "/" + name[comb_num] + ".log")
    train_list[comb_num] += train_command.string

In [12]:
print(train_command.string)
print("List length:", len(train_list))
index_to_show = 0
print("List command %d (%d/%d):\n%s" % (index_to_show, index_to_show+1, len(train_list), train_list[index_to_show]))


print("List command %d (%d/%d):\n%s" % (index_to_show, index_to_show+1, len(common_list), common_list[index_to_show]))


 --checkpoint /disks/md0-4T/users/b.mgm/checkpoints/training_71___char_rnn_50__rnn_dim_300__word_min_freq_10__train_batch_size_16  | tee -a /disks/md0-4T/users/b.mgm/logs/training_71___char_rnn_50__rnn_dim_300__word_min_freq_10__train_batch_size_16.log
List length: 72
List command 0 (1/72):
python /disks/md0-4T/users/b.mgm/model/train.py --data_path /disks/md0-4T/users/b.mgm/data/CRF --test_path test.bilou_CRF --val_path validation.bilou_CRF --train_path train.bilou_CRF --emb_path /disks/md0-4T/users/b.mgm/data/embedding/pretrained_embedding.txt --word_emb_dim 100 --val_batch_size 64 --char_emb_dim 25 --max_epochs 100 --char_rnn 25 --rnn_dim 100 --word_min_freq 0 --train_batch_size 4 --checkpoint /disks/md0-4T/users/b.mgm/checkpoints/training_0___char_rnn_25__rnn_dim_100__word_min_freq_0__train_batch_size_4  | tee -a /disks/md0-4T/users/b.mgm/logs/training_0___char_rnn_25__rnn_dim_100__word_min_freq_0__train_batch_size_4.log
List command 0 (1/72):
python /disks/md0-4T/users/b.mgm/model

## Train File

In [13]:
#Opening and writing in train file
train_file = open(json_list["output_train"], 'w')
for line in train_list:
    train_file.write(line + "\n")
train_file.close()

In [14]:
train_file = open(json_list["output_train"], 'r')
print("Train file:", json_list["output_train"], "\n\n", train_file.read())
train_file.close()

Train file: training_commands.lst 

 python /disks/md0-4T/users/b.mgm/model/train.py --data_path /disks/md0-4T/users/b.mgm/data/CRF --test_path test.bilou_CRF --val_path validation.bilou_CRF --train_path train.bilou_CRF --emb_path /disks/md0-4T/users/b.mgm/data/embedding/pretrained_embedding.txt --word_emb_dim 100 --val_batch_size 64 --char_emb_dim 25 --max_epochs 100 --char_rnn 25 --rnn_dim 100 --word_min_freq 0 --train_batch_size 4 --checkpoint /disks/md0-4T/users/b.mgm/checkpoints/training_0___char_rnn_25__rnn_dim_100__word_min_freq_0__train_batch_size_4  | tee -a /disks/md0-4T/users/b.mgm/logs/training_0___char_rnn_25__rnn_dim_100__word_min_freq_0__train_batch_size_4.log
python /disks/md0-4T/users/b.mgm/model/train.py --data_path /disks/md0-4T/users/b.mgm/data/CRF --test_path test.bilou_CRF --val_path validation.bilou_CRF --train_path train.bilou_CRF --emb_path /disks/md0-4T/users/b.mgm/data/embedding/pretrained_embedding.txt --word_emb_dim 100 --val_batch_size 64 --char_emb_dim 25